In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from typing import Dict,Tuple

In [2]:
# lets make the back propagation
def loss_gradients(forward_info:Dict[str,np.ndarray],
                  weight:Dict[str,np.ndarray])->Dict[str,np.ndarray]:
    dldp=-2*(forward_info["y"]-forward_info["p"])
    dpdn=np.ones_like(forward_info["n"])
    dpdb=np.ones_like(weight["b"])
    dndw=np.transpose(forward_info["x"],(1,0))
    dldn=dldp*dpdn
    
    dldw=np.dot(dndw,dldn)  # this is the gradient of the weight
    dldb=(dldp*dpdb).sum(axis=0) # this is the gradient of the bias

    loss_gradient:Dict[str,np.ndarray]={}  #initializing a dictonar
    loss_gradient["w"]=dldw
    loss_gradient["b"]=dldb
    return loss_gradient
    

In [3]:
def init_weight(n_cols:int)->Dict[str,np.ndarray]:
    ''' this iniitalizes the weights of the linear model'''
    weights:Dict[str,np.ndarray]={}
    w=np.random.randn(n_cols,1)
    b=np.random.randn(1,1)
    weights['w']=w
    weights['b']=b
    return weights

In [4]:
def permute_data(x:np.ndarray,
                y:np.ndarray):
    perm=np.random.permutation(x.shape[0])
    return x[perm],y[perm]

In [5]:
batch=Tuple[np.ndarray,np.ndarray]
def generate_batch(x:np.ndarray,
                    y:np.ndarray,
                    start:int=0,
                    batch_sz:int=10
                  )->batch:
    if start+batch_sz > x.shape[0]:
        batch_sz=x.shape[0]- start
    x_batch,y_batch=x[start:start+batch_sz],y[start:start+batch_sz]
    return x_batch,y_batch

In [6]:
def forward_loss(x:np.ndarray,
                y:np.ndarray,
                w:Dict[str,np.ndarray])->Tuple[Dict[str,np.ndarray],float]:
    
    n=np.dot(x,w['w'])
    p=n+w['b'] # this is kinda the prediction of this model BTW
    #y_reshaped=y.reshape(-1,1)
    loss=np.power((y-p),2)
    
    forward_info:Dict[str,np.ndarray]={}
    forward_info['x']=x
    forward_info['n']=n
    forward_info['p']=p
    forward_info['y']=y
    
    return forward_info,loss

In [7]:
def train(x:np.ndarray,
         y:np.ndarray,
         n_iter: int=10,
         learning_rate:float=0.01,
         batch_sz:int=32,
         return_loss:bool=False,
         return_weights:bool=False,
         seed: int=1) ->None:
    if seed:
        np.random.seed(seed)
    start=0  # the starting number that is going to be used in batching of data
    
    weight=init_weight(x.shape[1]) #initialize weight
    x,y=permute_data(x,y) # shuffle
    batch_sz=32

    if return_loss:
        losses=[]
        
    for i in range(n_iter):
        if start>x.shape[0]:
            start=0
        x_batch,y_batch=generate_batch(x,y,start,batch_sz)
        start+=batch_sz
        # lets train the model
        forward,loss=forward_loss(x,y,weight)
        
        if return_loss:
            losses.append(loss)  # append the loss
        # time for the backpropagation and the loss that has been calculated during the training
        loss_grad=loss_gradients(forward,weight)
        for key in weight.keys():
            weight[key]-=learning_rate*loss_grad[key]
    if return_weights:
        return loss,weight
    return None

# lets preprocess our dataset

In [8]:
from sklearn.datasets import fetch_california_housing
cali=fetch_california_housing()

In [9]:
cali

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [10]:
data=cali.data
target=cali.target

In [11]:
from sklearn.preprocessing import StandardScaler
s=StandardScaler()
data=s.fit_transform(data)

In [12]:
x_train,x_test,y_train,y_test=train_test_split(data,target,random_state=42)

In [13]:
x_train.shape,y_train.shape

((15480, 8), (15480,))

In [14]:
x_test.shape,y_test.shape

((5160, 8), (5160,))

In [15]:
y_train_arr=y_train.reshape((15480,1))
y_test_arr=y_test.reshape((5160,1))

# lets trin our model

In [16]:
train_info=train(x_train,
                y_train_arr,
                learning_rate=0.001,
                batch_sz=32,
                return_loss=True,
                return_weights=True,
                seed=23)
losses=train_info[0]
weights=train_info[1]

In [17]:
losses

array([[2.78010175e+31],
       [1.36711388e+31],
       [3.84828972e+28],
       ...,
       [3.79102589e+31],
       [4.04686958e+30],
       [3.07050970e+31]])

In [18]:
weights

{'w': array([[-4.83350747e+15],
        [ 2.43191851e+16],
        [ 4.03701763e+16],
        [ 3.31846345e+16],
        [-4.68366351e+16],
        [-1.35379343e+16],
        [ 1.64291107e+17],
        [-1.62198661e+17]]),
 'b': array([[1.62616982e+14]])}